## Build Larger Training Set

Now that this project looks somewhat promising. Let's build a proper dataset. The Steam API makes this extremely painful.

I don't ever want to have to do this again in my life, so we should try to pull down everything we could ever want and then just use that in the future.

In [1]:
import os
import json
import time
import datetime
import requests
from os.path import exists

In [2]:
if 'STEAM_API_KEY' not in os.environ:
  print("No API Key :(")
else:
  print("Found API Key.")
  STEAM_API_KEY = os.environ['STEAM_API_KEY']

Found API Key.


In [3]:
base_url = 'https://api.steampowered.com'

# The maximum number of matches the API will return at once.
num_matches_per_request = 100

def __request(method, path, **kwargs):
  url = base_url + path
  kwargs.setdefault('params', dict()).update(key=STEAM_API_KEY)

  max_tries = 5
  num_tries = 0
  while num_tries < max_tries:
    try:
      response = requests.request(method, url, **kwargs)
      if response.status_code == 429:
        print("429. Backing off.")
        print(response.headers)
        time.sleep(15)
        num_tries += 1
        continue
      response_json = response.json()
      return response_json
    except:
      print("Error on request.")
    
    num_tries += 1
    time.sleep(5)


def get_match_history_by_seq_num(seq_num, num_matches, **params):
  path = '/IDOTA2Match_570/GetMatchHistoryBySequenceNum/V001'
  params.update(start_at_match_seq_num=seq_num)
  params.update(matches_requested=num_matches)
  return __request('get', path, params=params)

def get_most_recent_matches():
  path = '/IDOTA2Match_570/GetMatchHistory/V001'
  return __request('get', path)

def get_most_recent_seq_number():
  response = get_most_recent_matches()
  most_recent_matches = response['result']
  
  max_seq_num = -1
  for match in most_recent_matches:
    seq_num = match['match_seq_num']
    max_seq_num = max(match_seq_num, max_seq_num)
    
  return max_seq_num


In [4]:
bulk_download_path = "dota_2_bulk.tsv"

if exists(bulk_download_path):
  print("Bulk file found. Scanning for last (largest) sequence number.")
  max_seq_num = -1
  with open(bulk_download_path, 'r') as bulk_file:
    for line in bulk_file:
        match_seq_num, _ = line.split('\t')
        max_seq_num = max(max_seq_num, int(match_seq_num))
        
  current_seq_num = max_seq_num

else:
  print("No Dota 2 Bulk File. Creating...")
  # Create the file, find the latest seq_num
  with open(bulk_download_path, 'w') as bulk_file:
    # Just create the file
    pass
  
  # We start at a sequence number from August 7, 2021 and work forward.
  current_seq_num = 5125114401
  
    
print("Starting at: ", current_seq_num)
num_iters = 0
max_seq_num = current_seq_num

with open(bulk_download_path, 'a') as bulk_file:
  while True:
    if num_iters % 100 == 0:
      print("Iterations: ", num_iters, ". seq_num: ", current_seq_num)

    # Don't hit the API too frequently.
    time.sleep(1.6)
    response = get_match_history_by_seq_num(current_seq_num, num_matches_per_request)

    if response is None:
      print("Skipping. Response was None...")
      continue

    matches = response['result']['matches']

    if len(matches) != num_matches_per_request:
        print("Problem. Expected: " + num_matches_per_request + " matches. Actual: ", len(matches))
        print(response)
        break

    for match in matches:
        if match['human_players'] != 10:
            # We only want "real" games of Dota so we're ignoring
            # games without 10 human players.
            continue
        try:
            match_seq_num = match['match_seq_num']
            
            # TODO(joshvarty)
            
            str_json = str(match)
            
            
            
            if "\t" in str_json:
              print("TAB DETECTED IN JSON!")
              print(str_json)
              continue
              
            if "\t" in match_seq_num:
              print("TAB DETECTED IN match_seq_num!")
              print(match_seq_num)
              continue
            
            bulk_file.write(str(match_seq_num))
            bulk_file.write('\t')
            bulk_file.write(str_json)
            bulk_file.write('\n')
            
            max_seq_num = max(max_seq_num, match_seq_num)

        except Exception as e:
            print(e)
            print("Error:", match)
            break

    current_seq_num = max_seq_num + 1
    num_iters += 1

print("Complete.")

Bulk file found. Scanning for last (largest) sequence number.
Starting at:  5125918630
Iterations:  0 . seq_num:  5125918630
Iterations:  100 . seq_num:  5125933224
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 06:27:42 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 06:27:42 GMT', 'Connection': 'keep-alive'}
Iterations:  200 . seq_num:  5125947527
Iterations:  300 . seq_num:  5125961980
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 06:33:45 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 06:33:45 GMT', 'Connection': 'keep-alive'}
Iterations:  400 . seq_num:  5125976327
Iterations:  500 . seq_num:  5125990812
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 06:40:52 GMT

Iterations:  2800 . seq_num:  5126307333
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 08:24:52 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 08:24:52 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 08:27:44 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 08:27:44 GMT', 'Connection': 'keep-alive'}
Iterations:  2900 . seq_num:  5126321040
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 08:29:39 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 08:29:39 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 

Iterations:  5200 . seq_num:  5126669022
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 09:57:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 09:57:49 GMT', 'Connection': 'keep-alive'}
Iterations:  5300 . seq_num:  5126684095
Iterations:  5400 . seq_num:  5126699256
Iterations:  5500 . seq_num:  5126714384
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 10:09:50 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 10:09:50 GMT', 'Connection': 'keep-alive'}
Iterations:  5600 . seq_num:  5126729559
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 10:13:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 10:13:51 GMT', 'Con

Iterations:  8000 . seq_num:  5127096275
Iterations:  8100 . seq_num:  5127111265
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 11:46:44 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 11:46:44 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 11:48:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 11:48:49 GMT', 'Connection': 'keep-alive'}
Iterations:  8200 . seq_num:  5127126044
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 11:52:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 11:52:49 GMT', 'Connection': 'keep-alive'}
Iterations:  8300 . seq_num:  5127140826
429. Backing off.

Iterations:  10700 . seq_num:  5127483747
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 13:29:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 13:29:51 GMT', 'Connection': 'keep-alive'}
Iterations:  10800 . seq_num:  5127497364
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 13:33:47 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 13:33:47 GMT', 'Connection': 'keep-alive'}
Iterations:  10900 . seq_num:  5127510900
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Sun, 01 May 2022 13:36:52 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Sun, 01 May 2022 13:36:52 GMT', 'Connection': 'keep-alive'}
Iterations:  11000 . seq_num:  5127524561
429. Backing 

...

429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 17:09:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 17:09:46 GMT', 'Connection': 'keep-alive'}
Iterations:  43100 . seq_num:  5132258855
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 17:12:50 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 17:12:50 GMT', 'Connection': 'keep-alive'}
Iterations:  43200 . seq_num:  5132271800
Iterations:  43300 . seq_num:  5132285231
Iterations:  43400 . seq_num:  5132298646
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 17:22:50 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 17:22:50 GMT', 'Connection': 'keep-alive'}
429. Backing 

429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 18:59:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 18:59:51 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 19:02:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 19:02:46 GMT', 'Connection': 'keep-alive'}
Iterations:  45200 . seq_num:  5132561972
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 19:04:48 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 19:04:48 GMT', 'Connection': 'keep-alive'}
Iterations:  45300 . seq_num:  5132577523
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires'

429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 20:44:42 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 20:44:42 GMT', 'Connection': 'keep-alive'}
Iterations:  47500 . seq_num:  5132921926
Iterations:  47600 . seq_num:  5132937301
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 20:48:50 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 20:48:50 GMT', 'Connection': 'keep-alive'}
Iterations:  47700 . seq_num:  5132952574
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 20:52:49 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 20:52:49 GMT', 'Connection': 'keep-alive'}
429. Backing off.
{'Content-Type': 'text/html; charset=

Iterations:  50000 . seq_num:  5133291928
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 22:40:46 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 22:40:46 GMT', 'Connection': 'keep-alive'}
Iterations:  50100 . seq_num:  5133306055
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Mon, 02 May 2022 22:43:51 GMT', 'Cache-Control': 'max-age=0, no-cache, no-store', 'Pragma': 'no-cache', 'Date': 'Mon, 02 May 2022 22:43:51 GMT', 'Connection': 'keep-alive'}
Iterations:  50200 . seq_num:  5133320249
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Error on request.
Skipping. Response was None...
429. Backing off.
{'Content-Type': 'text/html; charset=UTF-8', 'Content-Length': '108', 'Expires': 'Tue, 03 May 2022 01:10:47 GMT', 'Ca

KeyError: 'result'

Not sure what that last error was. We should update the script to handle that in the future. For now I have over 5 million games of Dota downloaded so I'm going to leave it.